# Exercise 5.3
# Alessandro Rizzi, Matteo Zortea & Marvin Wolff

Tasks a) b) c) d) e) f)

## a)
We Begin by writing down the XOR truth table: 
$$x1 \quad x2 \quad out$$
$$0 \quad 0 \quad 0$$
$$1 \quad 0 \quad 1$$
$$0 \quad 1 \quad 1$$
$$1 \quad 1 \quad 0$$
The XOR gate takes two binary inputs and returns a binary output. That means that the input is a vector of two elements. We can also see that the cases are limited to 4. Thus the training set is limited to a collection of 4 bisimensional vectors.

## b)
If we don't want to change $E_l$, we can just add a unitary bias to the input vector, thus incrementing its shape to $n=3$.

## c)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import nest
nest.set_verbosity("M_ERROR")
%matplotlib inline

ModuleNotFoundError: No module named 'nest'

Define neuron parameters and setup the dataset

In [ ]:
nparams = {'V_th': 0., 'V_reset': -1., 'tau_m': 40., 
           'tau_syn_ex': 5., 'tau_syn_in': 5., 
           't_ref': 0.5, 'E_L': -1.}

# define XOR data
data_inp = np.zeros((4, 2))
data_inp[1::2, 0] = 1
data_inp[2:, 1] = 1
data_cls = ((data_inp > 0.5).sum(axis=1) % 2) * 1
data_inp = np.hstack([data_inp, np.ones((len(data_inp), 1))])
print(data_inp)
print(data_cls)

In [ ]:
def run_feed_forward_network(inp, weights, maxrate=1000., 
                             duration=1000., nparams=nparams):
    """Execute a single feed forward network.
    
    Input:
        inp      array           either a single input vector or an 
                                 array of input vectors
        weights  list of arrays  list of the weight matrices between
                                 the different layers
        maxrate  float           rate of the spike sources that corresponds
                                 to input =1
        duration float           duration of the simulation per image
        nparams  dict            neuron parameters

    Output:
        spike_rates  list        list of the the spike rate arrays per layer
    """
    # Reset NEST
    nest.ResetKernel()

    # If only one input example is given: Put it into a 
    # (1, ninput) array so that the iteration gives only
    # 1 run
    if len(inp.shape) == 1:
        inp = inp.reshape(1, -1)

    # create spike sources
    num_inp = weights[0].shape[1]
    spikegenerators = nest.Create('poisson_generator', num_inp)

    # generate all neuron layers by iterating over the list of weights
    hiddens, spikedetectors = [], []
    for i in range(len(weights)):
        # create neurons for this layer and record their spikes
        neurons = weights[i].shape[0]
        hiddens.append(
            nest.Create('iaf_psc_exp', neurons, params=nparams)
        )
        spikedetectors.append(
            nest.Create('spike_recorder', neurons)
        )
        nest.Connect(hiddens[-1], spikedetectors[-1], 'one_to_one')

        # in the first layer get input from the spikesources, else from
        # the previous layer, which is the second to last in the hiddens
        # array
        if i == 0:
            presyn = spikegenerators
        else:
            presyn = hiddens[-2]
        # connect sources with appropriate weights
        nest.Connect(presyn, hiddens[-1], syn_spec={'weight': weights[i]})

    spike_rates = []
    for inpimg in inp:
        # set up simulation and equilibrate system
        nest.SetStatus(spikegenerators, {'rate': inpimg * maxrate})
        nest.Simulate(100.)
        # Reset spike counters
        for i in range(len(weights)):
            spikedetectors[i].n_events = 0

        # Do the actual simulation
        nest.Simulate(duration)

        # read out spikes
        spike_rates.append([])
        for i in range(len(weights)):
            spike_rates[-1].append(
                np.array(nest.GetStatus(spikedetectors[i], "n_events")) * 1000. / duration / maxrate
            )

    return spike_rates

We consider a three layer network, similar to the one shown below (the interface will give you all-too-all connectivity). Note that all activities are rescaled to the inputrate which you can set via the `maxrate` parameter. Compare this to the membrane and synaptic time constants choosen above.

![](xor_exercise.png)

In [ ]:
def run_xor(W_IH = np.array([[1., 0., 0.], [0., 1., 0.], [1., 1., -1.],]) * 100.,
            W_HL = np.array([[1, 1, -3],]) * 100.,
            maxrate = 1000.,
            duration = 1000.,
            ):
    """Execute the XOR network
    
    Inputs:
        W_IH     array   weight matrix from input to hidden layer
        W_HL     array   weight matrix from hidden to output layer
        maxrate  float   rate of the spike sources that corresponds
                         to input =1
    """
    fig, axes = plt.subplots(2, 1)
    
    # run the network for all inputs, defined above
    # (for later tasks you may want to make this a parameter)
    result = run_feed_forward_network(
        data_inp,
        [
            W_IH,
            W_HL,
        ],
        maxrate=maxrate,
        nparams=nparams
    )

    # plot the activities of both hidden (axes[0]) and output (axes[1]) layers
    # for the different points of the dataset (x-axis)
    for i in range(len(data_inp)):
        for j in range(len(result[i][0])):
            axes[0].plot([i], result[i][0][j], c=f"C{j}", marker='o', markersize=10)

        for j in range(len(result[i][1])):
            axes[1].plot([i], result[i][1][j], c=f"C{j}", marker='o', markersize=10)
        axes[1].plot(i, data_cls[i], c="black", marker='x', markersize=10)

    axes[0].set_title("hidden activities")
    axes[0].set_xticks(range(len(data_inp)))
    axes[0].set_xticklabels([str(d) for d in data_inp])
    axes[0].set_ylim(-0.05, 1.05)

    axes[1].set_title("label activities\n(black target, blue recorded rate/maxrate in SNN)")
    axes[1].set_xlabel("different inputs")
    axes[1].set_xticks(range(len(data_inp)))
    axes[1].set_xticklabels([str(d) for d in data_inp])
    axes[1].set_ylim(-0.05, 1.05)

    fig.tight_layout()
    print("result")
    pprint([res[0] for res in result])
    pprint([res[1] for res in result])

We now play a bit with the weigts scale to see how it affects the activities of the network:

In [ ]:
weight_scale = 1.
W_IH = weight_scale * np.array(
    [[1, 1, 1], 
     [1, 1, 1], 
     [1, 1, 1],
    ])
W_HL = weight_scale * np.array([[1, 1, 1],])
run_xor(W_IH, W_HL, maxrate=1000., duration=1000.)

In [ ]:
weight_scale = 10.
W_IH = weight_scale * np.array(
    [[1, 1, 1], 
     [1, 1, 1], 
     [1, 1, 1],
    ])
W_HL = weight_scale * np.array([[1, 1, 1],])
run_xor(W_IH, W_HL, maxrate=1000., duration=1000.)

In [ ]:
weight_scale = 100.
W_IH = weight_scale * np.array(
    [[1, 1, 1], 
     [1, 1, 1], 
     [1, 1, 1],
    ])
W_HL = weight_scale * np.array([[1, 1, 1],])
run_xor(W_IH, W_HL, maxrate=1000., duration=1000.)

In [ ]:
weight_scale = 50.
W_IH = weight_scale * np.array(
    [[1, 1, 1], 
     [1, 1, 1], 
     [1, 1, 1],
    ])
W_HL = weight_scale * np.array([[1, 1, 1],])
run_xor(W_IH, W_HL, maxrate=1000., duration=1000.)

We can see that increasing the weights scale we also increase the magnitude both of the hidden layer and of the final output. If we set it to a too small value, all the outputs will be null, while if we set it to a too large value, the outputs will be all over the targets. Thus we have to choose a reasonable intermediate value, that has to be adjusted with respect to the maxrate and to the different weights sets.

## d)
We will choose the simplest binarisation scheme for the output: 1 if the output $y$ is over a threshold $\Theta$ and 0 if $y \leq \Theta$. In our case, we will set $\Theta = 0.5$.
We then try using the weights from the previous sheet, and, adjusting a bit the scale, we obtain a corret classification:

In [ ]:
weight_scale = 150
W_IH = weight_scale * np.array(
    [[1, 0, 0], 
     [1, 1, -1], 
     [0, 1, 0],
    ])
W_HL = weight_scale * np.array([[1, -2, 1],])
run_xor(W_IH, W_HL, maxrate=1000., duration=1000.)

## e)

### i) AND gate:

In [ ]:
nparams = {'V_th': 0., 'V_reset': -1., 'tau_m': 40., 
           'tau_syn_ex': 5., 'tau_syn_in': 5., 
           't_ref': 0.5, 'E_L': -1.}

# define AND data
data_inp = np.zeros((4, 2))
data_inp[1::2, 0] = 1
data_inp[2:, 1] = 1
data_cls = [0,0,0,1]
data_inp = np.hstack([data_inp, np.ones((len(data_inp), 1))])
print(data_inp)
print(data_cls)

In [ ]:
def run_and(W_IH = np.array([[1., 0., 0.], [0., 1., 0.], [1., 1., -1.],]) * 100.,
            W_HL = np.array([[1, 1, -3],]) * 100.,
            maxrate = 1000.,
            duration = 1000.,
            ):
    """Execute the AND network
    
    Inputs:
        W_IH     array   weight matrix from input to hidden layer
        W_HL     array   weight matrix from hidden to output layer
        maxrate  float   rate of the spike sources that corresponds
                         to input =1
    """
    fig, axes = plt.subplots(2, 1)
    
    # run the network for all inputs, defined above
    # (for later tasks you may want to make this a parameter)
    result = run_feed_forward_network(
        data_inp,
        [
            W_IH,
            W_HL,
        ],
        maxrate=maxrate,
        nparams=nparams
    )

    # plot the activities of both hidden (axes[0]) and output (axes[1]) layers
    # for the different points of the dataset (x-axis)
    for i in range(len(data_inp)):
        for j in range(len(result[i][0])):
            axes[0].plot([i], result[i][0][j], c=f"C{j}", marker='o', markersize=10)

        for j in range(len(result[i][1])):
            axes[1].plot([i], result[i][1][j], c=f"C{j}", marker='o', markersize=10)
        axes[1].plot(i, data_cls[i], c="black", marker='x', markersize=10)

    axes[0].set_title("hidden activities")
    axes[0].set_xticks(range(len(data_inp)))
    axes[0].set_xticklabels([str(d) for d in data_inp])
    axes[0].set_ylim(-0.05, 1.05)

    axes[1].set_title("label activities\n(black target, blue recorded rate/maxrate in SNN)")
    axes[1].set_xlabel("different inputs")
    axes[1].set_xticks(range(len(data_inp)))
    axes[1].set_xticklabels([str(d) for d in data_inp])
    axes[1].set_ylim(-0.05, 1.05)

    fig.tight_layout()
    print("result")
    pprint([res[0] for res in result])
    pprint([res[1] for res in result])

In [ ]:
weight_scale = 150
W_IH = weight_scale * np.array(
    [[1, 1, -1.5], 
     [0, 0, -0], 
     [0, 0, 0],
    ])
W_HL = weight_scale * np.array([[1, -2, 1],])
run_and(W_IH, W_HL, maxrate=1000., duration=1000.)

### ii) OR gate:

In [ ]:
nparams = {'V_th': 0., 'V_reset': -1., 'tau_m': 40., 
           'tau_syn_ex': 5., 'tau_syn_in': 5., 
           't_ref': 0.5, 'E_L': -1.}

# define OR data
data_inp = np.zeros((4, 2))
data_inp[1::2, 0] = 1
data_inp[2:, 1] = 1
data_cls = [0,1,1,1]
data_inp = np.hstack([data_inp, np.ones((len(data_inp), 1))])
print(data_inp)
print(data_cls)

In [ ]:
def run_or(W_IH = np.array([[1., 0., 0.], [0., 1., 0.], [1., 1., -1.],]) * 100.,
            W_HL = np.array([[1, 1, -3],]) * 100.,
            maxrate = 1000.,
            duration = 1000.,
            ):
    """Execute the OR network
    
    Inputs:
        W_IH     array   weight matrix from input to hidden layer
        W_HL     array   weight matrix from hidden to output layer
        maxrate  float   rate of the spike sources that corresponds
                         to input =1
    """
    fig, axes = plt.subplots(2, 1)
    
    # run the network for all inputs, defined above
    # (for later tasks you may want to make this a parameter)
    result = run_feed_forward_network(
        data_inp,
        [
            W_IH,
            W_HL,
        ],
        maxrate=maxrate,
        nparams=nparams
    )

    # plot the activities of both hidden (axes[0]) and output (axes[1]) layers
    # for the different points of the dataset (x-axis)
    for i in range(len(data_inp)):
        for j in range(len(result[i][0])):
            axes[0].plot([i], result[i][0][j], c=f"C{j}", marker='o', markersize=10)

        for j in range(len(result[i][1])):
            axes[1].plot([i], result[i][1][j], c=f"C{j}", marker='o', markersize=10)
        axes[1].plot(i, data_cls[i], c="black", marker='x', markersize=10)

    axes[0].set_title("hidden activities")
    axes[0].set_xticks(range(len(data_inp)))
    axes[0].set_xticklabels([str(d) for d in data_inp])
    axes[0].set_ylim(-0.05, 1.05)

    axes[1].set_title("label activities\n(black target, blue recorded rate/maxrate in SNN)")
    axes[1].set_xlabel("different inputs")
    axes[1].set_xticks(range(len(data_inp)))
    axes[1].set_xticklabels([str(d) for d in data_inp])
    axes[1].set_ylim(-0.05, 1.05)

    fig.tight_layout()
    print("result")
    pprint([res[0] for res in result])
    pprint([res[1] for res in result])

In [ ]:
weight_scale = 100
W_IH = weight_scale * np.array(
    [[1, 1, 0], 
     [0, 0, 0], 
     [0, 0, 0],
    ])
W_HL = weight_scale * np.array([[1, -2, 1],])
run_or(W_IH, W_HL, maxrate=1000., duration=1000.)

### iii) NOT gate:
Differently from the previous tasks, here we deal with a monodimensional input:

In [ ]:
nparams = {'V_th': 0., 'V_reset': -1., 'tau_m': 40., 
           'tau_syn_ex': 5., 'tau_syn_in': 5., 
           't_ref': 0.5, 'E_L': -1.}

# define XOR data
data_inp = np.zeros((2, 1))
data_inp = [[0], [1]]
data_cls = [1, 0]
data_inp = np.hstack([data_inp, np.ones((len(data_inp), 1))])
print(data_inp)
print(data_cls)

In [ ]:
def run_not(W_IH = np.array([[1., 0., 0.], [0., 1., 0.], [1., 1., -1.],]) * 100.,
            W_HL = np.array([[1, 1, -3],]) * 100.,
            maxrate = 1000.,
            duration = 1000.,
            ):
    """Execute the NOT network
    
    Inputs:
        W_IH     array   weight matrix from input to hidden layer
        W_HL     array   weight matrix from hidden to output layer
        maxrate  float   rate of the spike sources that corresponds
                         to input =1
    """
    fig, axes = plt.subplots(2, 1)
    
    # run the network for all inputs, defined above
    # (for later tasks you may want to make this a parameter)
    result = run_feed_forward_network(
        data_inp,
        [
            W_IH,
            W_HL,
        ],
        maxrate=maxrate,
        nparams=nparams
    )

    # plot the activities of both hidden (axes[0]) and output (axes[1]) layers
    # for the different points of the dataset (x-axis)
    for i in range(len(data_inp)):
        for j in range(len(result[i][0])):
            axes[0].plot([i], result[i][0][j], c=f"C{j}", marker='o', markersize=10)

        for j in range(len(result[i][1])):
            axes[1].plot([i], result[i][1][j], c=f"C{j}", marker='o', markersize=10)
        axes[1].plot(i, data_cls[i], c="black", marker='x', markersize=10)

    axes[0].set_title("hidden activities")
    axes[0].set_xticks(range(len(data_inp)))
    axes[0].set_xticklabels([str(d) for d in data_inp])
    axes[0].set_ylim(-0.05, 1.05)

    axes[1].set_title("label activities\n(black target, blue recorded rate/maxrate in SNN)")
    axes[1].set_xlabel("different inputs")
    axes[1].set_xticks(range(len(data_inp)))
    axes[1].set_xticklabels([str(d) for d in data_inp])
    axes[1].set_ylim(-0.05, 1.05)

    fig.tight_layout()
    print("result")
    pprint([res[0] for res in result])
    pprint([res[1] for res in result])

In [ ]:
weight_scale = 100
W_IH = weight_scale * np.array(
    [[-1.5, 1]])
W_HL = weight_scale * np.array([[1],])
run_not(W_IH, W_HL, maxrate=1000., duration=1000.)

## f)
We just have to pass X1 and X2 to our trained XOR gate, and X3 and X4 to the AND gate. Then we pass the two outputs to the OR gate. If evry gate is trained separately, the training is stable. 